# Supervised Learning

This notebook is for supervised learning of the dataset, for comparison with
the UCL model in the Unsupervised notebook.

The expected input is a text file generated by the Preprocessing notebook.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.utils import ImageDataGenerator
import numpy as np
import utilities
import seaborn as sns

In [ ]:
#### os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [ ]:
labels = [0, 1, 2]
train_txt = []
valid_txt = []
test_txt = []

for i in labels:
    train_txt.append(f'data_txt/{i}_train.txt')
    valid_txt.append(f'data_txt/{i}_valid.txt')
    test_txt.append(f'data_txt/{i}_test.txt')

train_txt

In [ ]:
#train_paths_df = utilities.make_path_df(["grs_nogrs_Training.txt"])
train_paths_df = utilities.make_path_df(train_txt)
valid_paths_df = utilities.make_path_df(valid_txt)
test_paths_df = utilities.make_path_df(test_txt)

In [ ]:
# Check that it looks ok...
train_paths_df.head(5)

In [ ]:
#n = random.randint(0,len(all_paths_df))
#img = utilities.view_image(all_paths_df.iloc[n]["path"], all_paths_df.iloc[n]["label"], cmap="gray")
#print(all_paths_df.iloc[n]["path"])

In [ ]:
_ = utilities.view_images(test_paths_df, n_images=16, randomize=True).tight_layout()

In [ ]:
# Create train/validation/test sets of the paths dataframe, proportions 0.70/0.15/0.15
# Test set is used in evaluation AFTER training only.
# train_paths_df, valid_paths_df = train_test_split(all_paths_df, train_size=0.8, shuffle=True, random_state=4444)
# valid_paths_df, test_paths_df = train_test_split(test_paths_df, train_size=0.5, shuffle=True, random_state=4444)

# make datasets smaller for quicker training while developing:
#train_paths_df = train_paths_df[:int(len(train_paths_df)/4)]
#test_paths_df = test_paths_df[:int(len(test_paths_df)/4)]

In [ ]:
work_img_size = 112,112 # 224, 224 is default for most pretrained models
input_shape = work_img_size + (3,) # 3 for rgb (model is pretrained on rgb imgs)
batch_size = 16 # try decreasing this in case of out-of-memory errors for the GPU

img_datagen = ImageDataGenerator()

# Generators to import data from directories and turn it into batches.
train_data = img_datagen.flow_from_dataframe(train_paths_df,
                                               x_col='path',
                                               y_col='label',
                                               batch_size=batch_size,
                                               target_size=work_img_size,
                                               class_mode='categorical',
                                               validate_filenames=False,
                                               shuffle=True,
                                               seed=242)

valid_data = img_datagen.flow_from_dataframe(valid_paths_df,
                                               x_col='path',
                                               y_col='label',
                                               batch_size=batch_size,
                                               target_size=work_img_size,
                                               class_mode='categorical',
                                               validate_filenames=False,
                                               shuffle=False,
                                               seed=242)

test_data  =  img_datagen.flow_from_dataframe(test_paths_df,
                                               x_col='path',
                                               y_col='label',
                                               batch_size=batch_size,
                                               target_size=work_img_size,
                                               class_mode='categorical',
                                               validate_filenames=False,
                                               shuffle=False,
                                               seed=242)


In [ ]:
def do_transfer_learning(core_model, prepro_layer=None, weights="imagenet", cb=[],
                         input_shape=input_shape, freeze_layers=0, epochs=60):
    """

    :param core_model:
    :param prepro_layer:
    :param weights:
    :param cb:
    :param input_shape:
    :param freeze_layers:
    :param epochs:
    :return:
    """

    i = tf.keras.layers.Input(input_shape)
    core = core_model(weights=weights,
                      include_top=False,
                      input_tensor=i)
    x = core.output
    x = tf.keras.layers.Flatten(name='flatten')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(3, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=i, outputs=x)
    
    # Freeze some layers
    print("Freezing the first " + str(freeze_layers) + " layers (out of " + str(len(model.layers)) + ").")
    for nr, layer in enumerate(model.layers):
        if nr < freeze_layers:
            layer.trainable = False
        else:
            layer.trainable = True
    
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), # def lr is 0.01
                  loss=tf.keras.losses.BinaryCrossentropy(), 
                  metrics=['accuracy']
               )

    hist = model.fit(train_data,
                     verbose=2,
                     epochs=epochs, 
                     validation_data=valid_data,
                     callbacks=cb
                    )
                           
    return model, hist


In [ ]:
def eval_model(model, history, test_data=test_data):
    
    hist_df = pd.DataFrame(history.history)
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
    # fig.suptitle("Accuracy and Loss")
    ax1.plot(hist_df[["accuracy","val_accuracy"]])
    ax1.set_title("Accuracy")
    ax1.set_xlabel("epochs")
    ax1.set_ylim(top=1.0)

    ax2.plot(hist_df[["loss","val_loss"]])
    ax2.set_title("Loss")
    ax2.set_xlabel("epochs")
    ax2.set_ylim(bottom=0.0)

    fig.tight_layout()
    plt.tight_layout()
    plt.savefig("supervised_histories.png", dpi=300)
    #plt.show()
    
    y_probs = model.predict(test_data)
    y_preds = np.round(np.squeeze(y_probs)) # remove useless dimension
    y_preds = np.argmax(y_preds, axis=1)

    # get the true labels from the ImageDataGenerator
    y_true = test_data.classes

    classnames = list(test_data.class_indices.keys())
    cm = sklearn.metrics.confusion_matrix(y_true, y_preds)

    disp = sklearn.metrics.ConfusionMatrixDisplay(cm, 
                                           # y_true, 
                                                            # y_preds,
                                                            #normalize="all", # to get percentages
                                                            # cmap="Blues",
                                                            # colorbar=False,
                                                            display_labels=classnames)
    
    # print(cm)
    disp = disp.plot(cmap=plt.cm.Blues,values_format='g')
    plt.savefig('confusion_Matrix.png', dpi=300)
    plt.show()
    cr = utilities.make_classification_report(y_true, y_preds, classnames)
    print(cr)
    return y_probs, y_preds, disp

In [ ]:
# Make a few different callbacks
# Saves weights from the epoch with the lowest validation loss.
model_cpcb = tf.keras.callbacks.ModelCheckpoint(
                    filepath="best_checkpoint.ckpt",
                    verbose=1,
                    monitor='val_loss',
                    mode='min',
                    save_weights_only=True,
                    save_best_only=True)

# Stops training when loss is NaN. Should not normally happen.
nan_stopper = tf.keras.callbacks.TerminateOnNaN()

# Not used
class AccuracyEarlyStopperCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') > .99:
            print("\nReached >0.99 validation accuracy. Stopping training...\n")   
            self.model.stop_training = True
acc_stopper = AccuracyEarlyStopperCallback()

In [ ]:
model_vgg16, hist_vgg16 = do_transfer_learning(tf.keras.applications.vgg16.VGG16,
                                               cb=[nan_stopper, model_cpcb],
                                               input_shape=input_shape,
                                               freeze_layers=12,
                                               epochs=50)

In [ ]:
model_vgg16.load_weights("best_checkpoint.ckpt")

In [ ]:
pd.DataFrame(hist_vgg16.history)

In [ ]:
test_df = utilities.make_path_df(test_txt)
'''
test_data  =  img_datagen.flow_from_dataframe(test_df,
                                               x_col='path',
                                               y_col='label',
                                               batch_size=batch_size,
                                               target_size=work_img_size,
                                               class_mode='categorical',
                                               validate_filenames=False,
                                               shuffle=False,
                                               seed=242)
'''

In [ ]:
probs, preds, disp = eval_model(model_vgg16, hist_vgg16, test_data)

In [ ]:
# Display misclassified images

# get the true labels from the ImageDataGenerator
y_true = test_data.classes

res = pd.DataFrame({"path": test_df["path"], "prob":probs[:,0], "pred": preds.astype(int), "true":y_true})
res["correct"] = res["pred"] == res["true"]
misclassified = res[res["correct"] == False]
misclassified.to_csv('supervised_misclassified.csv')

In [ ]:
_ = utilities.view_images(misclassified["path"], labels=misclassified["prob"], n_images=64).tight_layout()